# Foundation

Before we can deep dive into tensor networks, we have to prepare 
ourselves with the tools needed to understand them. Our tool set 
will mostly contain mathematical notions and basic results. Why is
Mathematics so powerful and used in many other fields as 
Computer Science or Physics? One of the reasons is its built-in 
core concept of _abstraction_. It allows to start with very simple 
terms and combine them step by step into more complicated ones.

We will apply the same strategy. We start out with some thoughts about
abstractions and then introduce basic structures of Linear Algebra as 
_vector spaces_ and _linear maps_. Later we will generalize what we have 
learned here to define _tensors_ and _tensor networks_.

## Abstraction

We will use the programming language [Python](https://www.python.org/) 
for our coding examples. Recently it has undergone a raise in popularity.
Confer page
[About Python](https://executablebooks.github.io/quantecon-mini-example/docs/about_py.html),
also to gain more motivation and insights.
What are the reasons for this increase in interest?

Usually a programming language needs a killer app to be successful, or 
a killer framework. Ruby had Ruby on Rails, JavaScript's killer app is the 
web browser, Java was pushed by the JVM. According to the linked document, 
Python is the proven leader when it comes to data analysis, with 
[pandas](https://pandas.pydata.org/) being the library driving this effort.

Still, there are reasons within the language's design, enabling this wide acceptance.
Data analysis requires both, numerical performance to process huge data sets,
and expressive power to handle complex tasks and to have short development cycles.
Very often these demands stand for incompatible requirements, but Python got both 
right at the same time. With bindings for C/C++ code and even Fortran routines, it is 
able to consume high performant numerical code as incorporated in [NumPy](https://numpy.org/) 
for example. On the other hand, Python is long known for expressive syntax and 
concepts, for instance _list comprehensions_.

Combining both is a nice example of _abstraction_. The details of mathematical
algorithms are encapsulated in C/C++ assemblies and abstracted away from the 
user of libraries as NumPy or pandas. It feels very comfortable to use lightweight
glue code in the dynamically typed Python language, to combine the numerical 
building blocks into the solution of a real world problem. 

We will illustrate the power of abstraction on a much smaller scale 
(and completely in Python). We assume that our real world problem is to
print out all odd integers between `10` and `20`. This is not very realistic, 
but will serve the purpose of exemplification.

A very straight forward and not very abstract solution would be:

In [1]:
x = 11
while x < 20:
    print(x)
    x += 2

11
13
15
17
19


You might object that it would be a one-liner if we would have used the 
built-in [`range`](https://docs.python.org/3/library/stdtypes.html#range) 
generator. This is true, but on one hand this will be our starting point 
of a non-abstract solution, on the other hand if we would have had to 
use plain C code, the solution would have looked very similar to this example.

What is the problem with this snippet? It intertwines the part of generating
the numbers with the part of looping through the numbers with the part of 
printing the output. What if we would like to print out other content or 
use another output method? Much better would be to decouple these parts as 
in the following code, providing a solution to the same problem.

In [2]:
def log(print_fn, xs):
    for x in xs:
        print_fn(x)
        
log(print, range(11, 20, 2))

11
13
15
17
19


The parts are separated now. The number generation is provided by
`range`, the output done by `print`. The `log` function connects the 
low-level parts. It takes a function for printing and the numbers as iterable.
Both is combined using `for`, iterating through the numbers and feeding 
them into the print function.

What have we won? The glue code is simple, we could consider it 
to be declarative. Furthermore, the details are abstracted away. In `log`
there is no interest in how `print_fn` and `xs` are implemented. We just 
need to know that `print_fn` takes a number and outputs it, whereas `xs`
is iterable. This has the consequences, that under these requirements, 
the detail providing parts are easily exchangeable. `print_fn` could follow 
different output strategies, printing to stdout, or using an api-call to 
send it to a web service, or write it to a database. Also the content can 
easily be changed. The following snippet prints out all permutations of 
a given list, using 
[Heap's algorithm](https://en.wikipedia.org/wiki/Heap%27s_algorithm).

In [3]:
def permutations(xs):
    def heap(xs, k):
        def is_even(s):
            return s%2 == 0

        def swap(s, t):
            xs[s], xs[t] = xs[t], xs[s]

        if k == 1:            
            yield list(xs) # yield new list, not mutatable by next steps
        else:
            yield from heap(xs, k-1)
            for i in range(k-1):
                if is_even(k):
                    swap(i, k-1)
                else:
                    swap(0, k-1)
                yield from heap(xs, k-1)
                
    yield from heap(xs, len(xs))
    
log(print, permutations(["Penny", "Leonard", "Sheldon"]))

['Penny', 'Leonard', 'Sheldon']
['Leonard', 'Penny', 'Sheldon']
['Sheldon', 'Penny', 'Leonard']
['Penny', 'Sheldon', 'Leonard']
['Leonard', 'Sheldon', 'Penny']
['Sheldon', 'Leonard', 'Penny']


Having implemented `permutations`, we can now leave the detail level
behind us, and simple use it. A magic square is a square of numbers
with each row, column and both diagonals yielding the same sum.
Let us find all magic squares of numbers `1,...,9`. Using the existing
functions, this is now implemented in a few lines of code -
although not in the most performant way.

In [4]:
def check(numbers):
    patterns = [
        [0, 1, 2], # first row
        [3, 4, 5], # second row
        [0, 3, 6], # first column
        [1, 4, 7], # second column
        [0, 4, 8], # main diagonal
        [2, 4, 6], # secondary diagonal
    ]    
    for pattern in patterns:
        line = [num for i, num in enumerate(numbers) if i in pattern]
        if sum(line) != 15:
            return False        
    return True

def print_square(numbers):
    s = f"{numbers[0]} {numbers[1]} {numbers[2]}\n"
    s += f"{numbers[3]} {numbers[4]} {numbers[5]}\n"
    s += f"{numbers[6]} {numbers[7]} {numbers[8]}\n"
    print(s)

numbers = list(range(1, 10))
magic_squares = [p for p in permutations(numbers) if check(p)]

log(print_square, magic_squares)

4 9 2
3 5 7
8 1 6

4 3 8
9 5 1
2 7 6

6 1 8
7 5 3
2 9 4

6 7 2
1 5 9
8 3 4

8 1 6
3 5 7
4 9 2

8 3 4
1 5 9
6 7 2

2 7 6
9 5 1
4 3 8

2 9 4
7 5 3
6 1 8



To summarize, abstractions help to focus on the appropriate level 
of detail and to decompose complex structures into exchangable pieces.
Coming back to tensors, we will use exactly this strategy to develop 
our understanding. Basic building blocks are _vector spaces_ and 
_linear maps_. There is a huge variety of instances. Vector spaces can 
be based on real numbers, complex numbers, can contain arrays of fixed length,
infinite sequences or even continuous functions. Accordingly, linear maps 
can look quite differently as well. But to all these different objects,
the principles of linearity and preserving linearity are common.
Similarly to the mentioned considerations, we will leave out the details
of the specific instances and exploit linearity to obtain common properties.
This will lead to _tensors_ and _tensor networks_ as structures with a 
wide range of potential applications in Physics, Machine Learning and 
Data Science.

## Vector Spaces

We will fast forward through all necessary result of Linear Algebra.
To gain a more detailed view, we recommend the excellent text book
{cite}`axler15`.

## Linear Maps